# Covid Cases and Deaths Worldwide

### Source Data https://www.kaggle.com/datasets/themrityunjaypathak/covid-cases-and-deaths-worldwide

In [76]:
import pandas as pd
import pyodbc
import numpy as np
server = 'localhost' 
database = 'TestSSISDB' 
username = 'localhost' 
password = 'localhost'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

## Preprocessing Data

In [73]:
# Read Dataset from csv
df = pd.read_csv('covid_worldwide.csv')

# Replace NaN value to 0
df.fillna(0,inplace=True)

# Split between numeric datatype and character datatype
df_num = df.astype(str).apply(lambda x: x.str.replace(',','')).iloc[:,[0,2,3,4,5,6,7]].astype(int)
df_char = df.iloc[:,0:2]

# Merge between numeric and character datatype columns
df = df_char.merge(df_num,on='Serial Number',how='left')

# Show Dataframe Final
df

,Serial Number,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population
0,1,USA,104196861,1132935,101322779,1741147,1159832679,334805269
1,2,India,44682784,530740,44150289,1755,915265788,1406631776
2,3,France,39524311,164233,39264546,95532,271490188,65584518
3,4,Germany,37779833,165711,37398100,216022,122332384,83883596
4,5,Brazil,36824580,697074,35919372,208134,63776166,215353593
...,...,...,...,...,...,...,...,...
226,227,Diamond Princess,712,13,699,0,0,0
227,228,Vatican City,29,0,29,0,0,799
228,229,Western Sahara,10,1,9,0,0,626161
229,230,MS Zaandam,9,2,7,0,0,0


## Create Table Covid Cases in DB

In [77]:
cursor.execute(

"""
CREATE TABLE CovidCase (
    SerialNumber int primary key,
    Country nvarchar(50),
    TotalCases int,
    TotalDeaths int,
    TotalRecovered int,
    ActiveCases int,
    TotalTest int,
    Population int
)

"""

)

## Insert Dataframe to DB, and Power BI will Consumn this table

In [78]:
for row in df.itertuples():
    cursor.execute(
    
        """
        INSERT INTO CovidCase (
            SerialNumber,
            Country,
            TotalCases,
            TotalDeaths,
            TotalRecovered,
            ActiveCases,
            TotalTest,
            Population
        )
        VALUES (?,?,?,?,?,?,?,?)
        """,
        row._1,
        row.Country,
        row._3,
        row._4,
        row._5,
        row._6,
        row._7,
        row.Population
    
    
    )

In [82]:
cnxn.commit()